In [2]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jun 11 11:02:03 2021

@author: Christian
"""

from mip import Model, xsum, maximize, BINARY

from Co2_price import df_data
from electricity_demand import electricity_demand
from Electricity_Price0 import Electricity_price_pivot0
from Electricity_Price1 import Electricity_price_pivot1
from gas_price import gas_p
from heat_demand import heat_demand_pivot
from heat_price import heat_p
from elec_effciency import elec_eff
from electricity_capacity import elec_capacity
from heat_capacity import heat_capacity


elec_demand = electricity_demand.values.tolist()
elec_price_wholesale = Electricity_price_pivot0
elec_price_household_32_05 = Electricity_price_pivot1
co2_price = df_data
gas_price = gas_p
heat_demand_1 = heat_demand_pivot
heat_price_1 = heat_p


el_price = elec_price_wholesale.values.tolist() # Electricity price
del_t = .25 # duration of time step
el_demand = electricity_demand.values.tolist() # Electicity demand


gas_pp = gas_price["Preis"].values.tolist() # Gas price for different plants
em_fc = 0.2 # emission factor
co2_p = co2_price["mean_CO2_tax"].values.tolist() # CO2 Price
capacity_el = elec_capacity.values.tolist() # maximum capacity of electricity for the power plants
capacity_ht = heat_capacity.values.tolist() # maximum capacity of heat for the power plants
heat_demand = heat_demand_1.values.tolist() # heat demand
heat_price = heat_price_1.values.tolist() # heat price
heat_ratio = 400/385 # heat ratio


eff_plants = elec_eff.values.tolist() # the efficiency of the power plants


T = range(len(el_price))
I = range(len(gas_pp))

m = Model("Maximizing profit", sense=maximize)
 

#variable
y_t = [[m.add_var(lb = 0) for i in I] for t in T] # Electricity generation
el_sold = [[m.add_var(lb = 0) for i in I] for t in T] # electricity sold 
# el_sold = [m.add_var(lb = 0) for t in T] # electricity sold 
x_t = [[m.add_var(lb = 0) for i in I] for t in T] # Fuel consumption
z_t = [[m.add_var(lb = 0) for i in I] for t in T] # Heat generation
# objective function

# Revenue = Electricity price * demand/generation
# Cost = Fuel price = gas price + emission_factor * co2 price

# Max Proft = Revenue - Cost




m.objective = xsum((el_price[t] * del_t * el_sold[t]) - ( x_t[t][i] * ( gas_p[i] * del_t + (em_fc * co2_p[t]*del_t))) for t in T for i in I) + ((heat_demand[t] * heat_price[t]) for t in T)


m.objective = xsum((el_price[t][i] * del_t * el_sold[t][i]) - (x_t[t][i] * ( gas_p[i] * del_t + (em_fc * co2_p[t]*del_t))) for t in T for i in I) + ((heat_demand[t][i] * heat_price[t][i]) for t in T for i in I)


m.objective = xsum((el_price[t] * del_t * el_sold[t]) - ( x_t[t][i] * ( gas_p[i] * del_t + (em_fc * co2_p[t]*del_t))) for t in T for i in I) + ((heat_demand[t] * heat_price[t]) for t in T)

# constraints

for t in T:
    m += xsum(y_t[t][i] for i in I) == el_demand[t]
    
    for i in I:
        m += y_t[t][i] <= capacity_el[i] # electricity generation <= maximum capacity of electricity of the plant

        m += z_t[t][i] <= capacity_ht[i] # heat generation <= maximum capacity of heat of the plant 
        
        m += x_t[t][i] == y_t[t][i]/eff_plants[i] # fuel consumption = Electricity generation / efficiency of the plants 

        m += heat_demand[t] <= z_t[t][i] # heat demand >= Heat generation  

        m += y_t[t][i] == heat_ratio * z_t[t][i] # electricity generation >= heat ratio * heat generation

        m += y_t[t][i] == el_sold[t] + el_demand[t]



status = m.optimize()
obj= m.objective_value

status




Co2 Price

            mean_CO2_tax
Datum                   
2020-01-01         17.43
2020-01-02         17.43
2020-01-03         17.43
2020-01-04         17.43
2020-01-05         19.93
electricity demand

            00:00:00  01:00:00  02:00:00  03:00:00  04:00:00  05:00:00  \
Date                                                                     
2020-01-01    63.975    54.150    49.075    48.625    56.700    65.400   
2020-01-02    63.975    54.150    49.075    48.625    56.700    65.400   
2020-01-03    63.975    54.150    49.075    48.625    56.700    65.400   
2020-01-04    71.075    75.450    66.275    63.625    65.350    69.275   
2020-01-05    60.500    54.525    47.825    42.900    41.875    43.100   

            06:00:00  07:00:00  08:00:00  09:00:00  ...  14:00:00  15:00:00  \
Date                                                ...                       
2020-01-01    75.400   103.625   168.375   225.850  ...   172.050   185.200   
2020-01-02    75.400   103.625   168.3